# import libraries

In [270]:
import ee
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Authentication and Initialization  -
Before you can make requests to Earth Engine through a client library, you must authenticate

In [272]:
ee.Authenticate()

True

In [273]:
ee.Initialize()

# Get statistics of Rectangels(regions) around the specific points

In [275]:
#dowenload the image collection and filter it with date and get the median image
dataset = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterDate('2022-09-01', '2022-09-30').median()  

In [276]:
# Define the points (coordinates)
points = [
    [30.488472, 31.173472],
    [30.524722, 31.151861],
    [30.566028, 31.130972],
    [30.647556, 31.077806],
    [30.734028, 31.028500],
    [30.807556, 31.013361],
    [30.790639, 30.965194],
    [30.801278, 30.913278],
    [30.815306, 30.904389],
    [30.848333, 30.901222],
    [30.876917, 30.873500],
    [30.899056, 30.856250],
    [30.930333, 30.835639],
    [30.961583, 30.783083],
    [31.003111, 30.759333],
    [31.043556, 30.735694],
    [31.085667, 30.727861],
    [31.101222, 30.683639],
    [31.143611, 30.645361],
    [31.182500, 30.600361],
    [31.207500,30.569444],
    [31.224444,30.531583],
    [31.264361,30.518389],
    [31.311250,30.515528],
    [31.344417,30.482861],
    [30.851444,31.030083],
    [30.894389,31.024361],
    [30.893667,31.024667],
    [30.931611,30.990083],
    [30.980056,30.970417],
    [31.031639,30.961556],
    [31.085750,30.965417],
    [31.142111,30.968861],
    [31.214722,30.984111],
    [31.268806,30.990528],
    [31.261778,31.049611],
    [31.278194,31.093417],
    [31.328944,31.132139],
    [31.399000,31.125000],
    [31.471750,31.126861],
    [31.526444,31.140306],
    [31.565361,31.081889],
    [31.586472,31.052500],
    [31.412167,30.451000],
    [31.340417,30.366278],
    [31.205000,30.091444],
    [31.167278,30.069167],
    [31.128167,30.102194],
    [31.120861,30.164528],
    [31.108250,30.211944],
    [31.086417,30.294417],
    [31.069194,30.359806],
    [31.013250,30.473583],
]

In [277]:
# Create a feature collection from the points
features = ee.FeatureCollection([ee.Feature(ee.Geometry.Point(coord)) for coord in points])

In [278]:
# Define the size of the rectangle in meters (1 km x 1 km)
size = 7000  # Size in meters

In [279]:
# Create a function to create a rectangle around a point
def create_rectangle(point):
    lat = point[1]
    lon = point[0]
    
    # Calculate the corners of the rectangle
    topLeft = [
        lon - (size / 20037508.34 * 180 / 3.141592653589793),
        lat + (size / 20037508.34 * 180 / 3.141592653589793)
    ]
    
    bottomRight = [
        lon + (size / 20037508.34 * 180 / 3.141592653589793),
        lat - (size / 20037508.34 * 180 / 3.141592653589793)
    ]
    
    # Return the rectangle geometry
    return ee.Geometry.Rectangle(
        topLeft[0], bottomRight[1],
        bottomRight[0], topLeft[1]
    )

In [280]:
# Create a feature collection of rectangles
rectangles = ee.FeatureCollection([ee.Feature(create_rectangle(pt)) for pt in points])

In [281]:
# Function to get statistics for each rectangel
def get_statistics(feature):
    point = feature.geometry()
    # Calculate the median for each band in the median image
    stats = dataset.reduceRegion(
        reducer=ee.Reducer.median(),
        geometry=point,
        scale=30  # resolution in meters
    )
    return feature.set(stats)

In [282]:
# Apply the function to each rectangel in the features collection
stats_features = rectangles.map(get_statistics)

In [283]:
# Extract features and their statistics from the server-side
features = stats_features.getInfo().get('features', [])

# prepare the table data to store the statistics results

In [ ]:
# Initialize a data table as a dictionary
data_table = {
    'Point Longitude': [],
    'Point Latitude': [],
    'SR_B1': [],
    'SR_B2': [],
    'SR_B3': [],
    'SR_B4': [],
    'SR_B5': [],
    'SR_B6': [],
    'SR_B7': []
}
# Loop through features and populate the data table
for feature in features:
    properties = feature['properties']
    coords = feature['geometry']['coordinates'][0]  # Assuming the coordinates are in the first list
    
    # Calculate the centroid manually since Python does not have direct access like JS
    centroid_lon = sum([coord[0] for coord in coords]) / len(coords)
    centroid_lat = sum([coord[1] for coord in coords]) / len(coords)

    # Populate data_table with the centroid coordinates and properties
    data_table['Point Longitude'].append(centroid_lon)
    data_table['Point Latitude'].append(centroid_lat)
    data_table['SR_B1'].append(properties.get('SR_B1'))
    data_table['SR_B2'].append(properties.get('SR_B2'))
    data_table['SR_B3'].append(properties.get('SR_B3'))
    data_table['SR_B4'].append(properties.get('SR_B4'))
    data_table['SR_B5'].append(properties.get('SR_B5'))
    data_table['SR_B6'].append(properties.get('SR_B6'))
    data_table['SR_B7'].append(properties.get('SR_B7'))

In [285]:
#Create a DataFrame to display statistics
df = pd.DataFrame(data_table)

# Print the DataFrame
print(df)

# Save the result to a CSV file
csv_filename = 'land9rec_statistics.csv'
df.to_csv(csv_filename, index=False)
print(f'Results saved to {csv_filename}')

    Point Longitude  Point Latitude         SR_B1         SR_B2         SR_B3  \
0         30.484469       31.169469   8542.257552   8990.842453  10335.968933   
1         30.520719       31.147858   8672.115410   9183.551312  10463.319309   
2         30.562025       31.126969   8544.434922   8990.376096  10255.755113   
3         30.643553       31.073803   8591.283191   8976.389769  10032.113899   
4         30.730025       31.024497   8399.242974   8751.903998   9950.633029   
5         30.803553       31.009358   8463.304379   8879.343494   9999.528547   
6         30.786636       30.961191   8527.543621   8943.305339   9903.664434   
7         30.797275       30.909275   8478.800231   8927.538575   9951.134365   
8         30.811303       30.900386   8478.962482   8864.170318   9950.581396   
9         30.844330       30.897219   8559.565634   8912.089777   9967.351951   
10        30.872914       30.869497   8623.895203   8975.427220   9999.751366   
11        30.895053       30

# calculate the correlation between bands and HM

In [287]:
# read data and display it 
data=pd.read_csv('land9rec_statistics.csv')
data.head()

,Point Longitude,Point Latitude,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7
0,30.484469,31.169469,8542.257552,8990.842453,10335.968933,10271.106088,19295.000631,14047.413475,11232.601746
1,30.520719,31.147858,8672.115410,9183.551312,10463.319309,10526.645892,18623.800642,14270.421080,11617.215260
2,30.562025,31.126969,8544.434922,8990.376096,10255.755113,10271.003404,18144.426569,13982.809119,11294.156589
3,30.643553,31.073803,8591.283191,8976.389769,10032.113899,10254.651182,16288.806148,13406.079568,11230.335729
4,30.730025,31.024497,8399.242974,8751.903998,9950.633029,9760.066596,16833.159048,12767.169684,10464.016436


In [288]:
# add the HM concentration column to df
data['HM']=[
384.9,
318.3,
342.4,
440.8,
397.6,
380.4,
381.2,
414.5,
333.9,
362.8,
366.9,
407.6,
373.5,
382.0,
385.6,
356.8,
293.4,
332.1,
363.2,
429.9,
378.6,
419.2,
399.6,
189.0,
427.1,
429.2,
392.4,
387.6,
437.0,
444.5,
410.6,
453.4,
409.2,
448.0,
376.7,
395.0,
397.7,
347.8,
328.7,
292.1,
222.7,
152.0,
141.5,
287.4,
340.4,
318.1,
309.6,
325.8,
246.8,
231.3,
326.1,
276.7,
340.9]

In [289]:
data.head()

,Point Longitude,Point Latitude,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7,HM
0,30.484469,31.169469,8542.257552,8990.842453,10335.968933,10271.106088,19295.000631,14047.413475,11232.601746,384.9
1,30.520719,31.147858,8672.115410,9183.551312,10463.319309,10526.645892,18623.800642,14270.421080,11617.215260,318.3
2,30.562025,31.126969,8544.434922,8990.376096,10255.755113,10271.003404,18144.426569,13982.809119,11294.156589,342.4
3,30.643553,31.073803,8591.283191,8976.389769,10032.113899,10254.651182,16288.806148,13406.079568,11230.335729,440.8
4,30.730025,31.024497,8399.242974,8751.903998,9950.633029,9760.066596,16833.159048,12767.169684,10464.016436,397.6


In [290]:
# split the target(HM) and get correlation between it and features(bands)
target='HM'
#pearson correlation
corr=data.corr()[target]
print(corr)

Point Longitude   -0.457153
Point Latitude     0.234361
SR_B1             -0.231367
SR_B2             -0.251762
SR_B3             -0.353670
SR_B4             -0.291042
SR_B5             -0.334104
SR_B6             -0.508403
SR_B7             -0.329160
HM                 1.000000
Name: HM, dtype: float64
